In [ ]:
# 환경 설정 및 라이브러리 임포트
import asyncio
import json
import logging
import os
import sys
import time
from datetime import datetime
from pathlib import Path

# 환경변수 로딩
from dotenv import load_dotenv
load_dotenv()

from typing import Dict, List, Any, Optional
import httpx
from dataclasses import dataclass
import pandas as pd

# 로깅 설정
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s",
    handlers=[
        logging.StreamHandler(sys.stdout),
        logging.FileHandler("rag_pipeline.log")
    ]
)
logger = logging.getLogger(__name__)

# 프로젝트 루트 디렉토리 설정
PROJECT_ROOT = Path.cwd()
DOCS_ROOT = PROJECT_ROOT / "docs"
RESULTS_DIR = PROJECT_ROOT / "results"
PROCESSED_DIR = PROJECT_ROOT / "processed"

# 결과 저장 디렉토리 생성
RESULTS_DIR.mkdir(exist_ok=True)
PROCESSED_DIR.mkdir(exist_ok=True)

print("✅ 환경 설정 완료")
print(f"📁 프로젝트 루트: {PROJECT_ROOT}")
print(f"📁 문서 폴더: {DOCS_ROOT}")
print(f"📁 결과 폴더: {RESULTS_DIR}")
print(f"📁 처리 폴더: {PROCESSED_DIR}")


In [ ]:
# 문서 타입 매핑 및 데이터 클래스 정의

# 문서 타입 매핑 (폴더 경로 -> 문서 타입)
DOCUMENT_TYPE_MAPPING = {
    "근거 자료/법령": "law",  # 법령 파일들
    "근거자료/법령": "law",  # 법령 파일들 (공백 없는 버전)
    "근거 자료/체결계약": "executed_contract",  # 체결된 계약서들
    "근거자료/체결계약": "executed_contract",  # 체결된 계약서들 (공백 없는 버전)
    "근거 자료/표준계약서": "standard_contract",  # 표준 계약서 템플릿들
    "근거자료/표준계약서": "standard_contract",  # 표준 계약서 템플릿들 (공백 없는 버전)
    "1. NDA": "standard_contract",  # NDA 표준계약서 (기존 폴더)
    "2. 제품(서비스) 판매": "standard_contract",  # 판매계약서 표준 (기존 폴더)
}


@dataclass
class ProcessingResult:
    """문서 처리 결과를 담는 데이터 클래스"""
    filename: str
    doc_type: str
    folder_path: str
    success: bool
    processing_time: float
    page_count: Optional[int] = None
    chunk_count: Optional[int] = None
    error_message: Optional[str] = None
    doc_parser_result: Optional[Dict] = None

print("✅ 데이터 클래스 정의 완료")


In [ ]:
# ExternalServiceClient 클래스 정의 (완전한 클래스)

class ExternalServiceClient:
    """외부 서비스 클라이언트 통합 클래스"""
    
    def __init__(self):
        self.doc_converter_url = os.getenv("DOC_CONVERTER_URL", "http://localhost:8001")
        self.doc_parser_url = os.getenv("DOC_PARSER_URL", "http://localhost:8002")
        self.api_url = os.getenv("API_URL", "http://localhost:8000")
        self.timeout = 600.0
    
    async def health_check_all(self) -> Dict[str, bool]:
        """모든 서비스의 헬스체크"""
        services = {
            "doc-converter": f"{self.doc_converter_url}/health",
            "doc-parser": f"{self.doc_parser_url}/health",
            "api": f"{self.api_url}/health"
        }
        
        results = {}
        async with httpx.AsyncClient(timeout=10.0) as client:
            for service, url in services.items():
                try:
                    response = await client.get(url)
                    results[service] = response.status_code == 200
                    logger.info(f"✅ {service} 서비스 정상")
                except Exception as e:
                    results[service] = False
                    logger.error(f"❌ {service} 서비스 오류: {e}")
        
        return results
    
    def determine_doc_type(self, file_path: Path) -> str:
        """파일 경로를 기반으로 문서 타입 결정"""
        # 상대 경로 계산 (docs 폴더 기준)
        relative_path = file_path.relative_to(DOCS_ROOT)
        folder_path = str(relative_path.parent)
        
        # 폴더 경로 매핑에서 문서 타입 찾기
        for pattern, doc_type in DOCUMENT_TYPE_MAPPING.items():
            if folder_path.startswith(pattern) or folder_path == pattern:
                return doc_type
        
        # 기본값: law (근거자료)
        return "law"

    async def search_documents_direct(self, query: str, top_k: int = 5, doc_types: List[str] = None) -> Dict:
        """데이터베이스 직접 접근으로 문서 검색"""
        try:
            # 직접 임베딩 서비스 및 검색 로직 구현
            from sqlmodel.ext.asyncio.session import AsyncSession
            from sqlalchemy.ext.asyncio import create_async_engine
            from src.aws.embedding_service import TitanEmbeddingService
            from sqlalchemy import text
            
            # 데이터베이스 연결
            database_url = os.getenv("DATABASE_URL") or f"postgresql+asyncpg://{os.getenv('DATABASE_USER', 'postgres')}:{os.getenv('DATABASE_PASSWORD', 'postgres')}@{os.getenv('DATABASE_HOST', 'localhost')}:{os.getenv('DATABASE_PORT', '5434')}/{os.getenv('DATABASE_NAME', 'smartclm-poc')}"
            async_engine = create_async_engine(database_url, echo=False)
            
            # 임베딩 서비스 초기화
            embedding_service = TitanEmbeddingService()
            
            async with AsyncSession(async_engine) as session:
                # 1. 쿼리 임베딩 생성
                query_embedding = await embedding_service.create_single_embedding(query)
                
                # 2. 벡터 검색 쿼리 구성
                base_query = """
                SELECT 
                    c.id as chunk_id,
                    c.content,
                    c.chunk_type,
                    c.parent_id,
                    d.id as document_id,
                    d.filename,
                    d.doc_type,
                    d.category,
                    (1 - (c.embedding <=> :query_embedding)) as similarity_score
                FROM chunks c
                JOIN documents d ON c.document_id = d.id
                WHERE c.embedding IS NOT NULL
                AND d.room_id IS NULL 
                """
                
                # 벡터를 pgvector 형식 문자열로 변환
                vector_str = "[" + ",".join(map(str, query_embedding)) + "]"
                params = {"query_embedding": vector_str}
                
                # 문서 유형 필터
                if doc_types:
                    type_conditions = []
                    for i, doc_type in enumerate(doc_types):
                        param_name = f"doc_type_{i}"
                        type_conditions.append(f"d.doc_type = :{param_name}")
                        params[param_name] = doc_type
                    base_query += f" AND ({' OR '.join(type_conditions)})"
                
                # 유사도 정렬 및 제한
                base_query += " ORDER BY similarity_score DESC LIMIT :top_k"
                params["top_k"] = top_k
                
                # 3. 쿼리 실행
                connection = await session.connection()
                result = await connection.execute(text(base_query), params)
                rows = result.fetchall()
                
                # 4. 결과 변환
                search_results = []
                for row in rows:
                    search_results.append({
                        "chunk_id": row.chunk_id,
                        "content": row.content,
                        "similarity_score": round(row.similarity_score, 4),
                        "chunk_type": row.chunk_type,
                        "parent_id": row.parent_id,
                        "document_id": row.document_id,
                        "filename": row.filename,
                        "doc_type": row.doc_type,
                        "category": row.category,
                    })
                
                return {
                    "results": search_results,
                    "total_found": len(search_results),
                    "query": query
                }
                
        except Exception as e:
            logger.error(f"❌ 직접 검색 실패: {str(e)}")
            return {"results": [], "total_found": 0, "error": str(e)}

print("✅ ExternalServiceClient 기본 메서드 정의 완료")


In [ ]:
# ExternalServiceClient 클래스 확장 메서드들

def add_client_methods(client_class):
    """ExternalServiceClient에 추가 메서드들을 동적으로 추가"""
    
    async def convert_to_pdf_if_needed(self, file_path: Path) -> Path:
        """필요시 PDF로 변환 (DOCX 등)"""
        if file_path.suffix.lower() == '.pdf':
            return file_path
        
        logger.info(f"🔄 PDF 변환 중: {file_path.name}")
        
        try:
            async with httpx.AsyncClient(timeout=self.timeout) as client:
                with open(file_path, "rb") as f:
                    files = {
                        "file": (file_path.name, f, "application/octet-stream")
                    }
                    
                    response = await client.post(
                        f"{self.doc_converter_url}/convert",
                        files=files
                    )
                
                if response.status_code == 200:
                    # PDF 파일을 임시 저장
                    pdf_path = PROCESSED_DIR / f"{file_path.stem}_converted.pdf"
                    with open(pdf_path, "wb") as f:
                        f.write(response.content)
                    
                    logger.info(f"✅ PDF 변환 완료: {pdf_path.name}")
                    return pdf_path
                else:
                    raise Exception(f"변환 실패: HTTP {response.status_code}")
                    
        except Exception as e:
            logger.error(f"❌ PDF 변환 실패: {str(e)}")
            raise

    async def analyze_document_file(self, file_path: Path) -> ProcessingResult:
        """문서 파일을 분석"""
        start_time = time.time()
        doc_type = self.determine_doc_type(file_path)
        relative_path = file_path.relative_to(DOCS_ROOT)
        folder_path = str(relative_path.parent)
        
        try:
            logger.info(f"📄 문서 분석 시작: {file_path.name} (타입: {doc_type})")
            
            # PDF로 변환 (필요시)
            pdf_file_path = await self.convert_to_pdf_if_needed(file_path)
            
            async with httpx.AsyncClient(timeout=self.timeout) as client:
                with open(pdf_file_path, "rb") as f:
                    files = {
                        "file": (pdf_file_path.name, f, "application/pdf")
                    }
                    data = {"smart_pipeline": True}
                    
                    response = await client.post(
                        f"{self.doc_parser_url}/analyze",
                        files=files,
                        data=data
                    )
                
                if response.status_code == 200:
                    result = response.json()
                    processing_time = time.time() - start_time
                    
                    return ProcessingResult(
                        filename=file_path.name,
                        doc_type=doc_type,
                        folder_path=folder_path,
                        success=True,
                        processing_time=processing_time,
                        page_count=result.get("page_count"),
                        chunk_count=len(result.get("chunks", [])),
                        doc_parser_result=result
                    )
                else:
                    error_msg = f"Doc Parser 오류 (HTTP {response.status_code}): {response.text}"
                    logger.error(error_msg)
                    return ProcessingResult(
                        filename=file_path.name,
                        doc_type=doc_type,
                        folder_path=folder_path,
                        success=False,
                        processing_time=time.time() - start_time,
                        error_message=error_msg
                    )
            
            # 임시 PDF 파일 정리
            if pdf_file_path != file_path and pdf_file_path.exists():
                pdf_file_path.unlink()
                    
        except Exception as e:
            error_msg = f"문서 분석 실패: {str(e)}"
            logger.error(error_msg)
            return ProcessingResult(
                filename=file_path.name,
                doc_type=doc_type,
                folder_path=folder_path,
                success=False,
                processing_time=time.time() - start_time,
                error_message=error_msg
            )

    # 메서드들을 클래스에 추가
    client_class.convert_to_pdf_if_needed = convert_to_pdf_if_needed
    client_class.analyze_document_file = analyze_document_file
    return client_class

# ExternalServiceClient에 메서드 추가
ExternalServiceClient = add_client_methods(ExternalServiceClient)
print("✅ ExternalServiceClient 변환 및 분석 메서드 추가 완료")


In [ ]:
# ExternalServiceClient save_to_database 메서드 추가

def add_save_method(client_class):
    """save_to_database 메서드 추가"""
    
    async def save_to_database(self, parsed_result: Dict, source_file: Path, doc_type: str) -> bool:
        """파싱된 결과를 직접 데이터베이스에 저장 (S3 업로드 없음)"""
        try:
            logger.info(f"💾 데이터베이스 저장 시작: {source_file.name} (타입: {doc_type})")
            
            # 로컬에서 직접 데이터베이스에 저장
            from datetime import datetime
            from sqlmodel.ext.asyncio.session import AsyncSession
            from sqlalchemy.ext.asyncio import create_async_engine
            from src.models import Document, Chunk
            from src.aws.embedding_service import TitanEmbeddingService
            from src.documents.chunking import HierarchicalChunker
            import uuid
            
            # 직접 환경변수에서 데이터베이스 URL 가져오기
            database_url = os.getenv("DATABASE_URL") or f"postgresql+asyncpg://{os.getenv('DATABASE_USER', 'postgres')}:{os.getenv('DATABASE_PASSWORD', 'postgres')}@{os.getenv('DATABASE_HOST', '127.0.0.1')}:{os.getenv('DATABASE_PORT', '5434')}/{os.getenv('DATABASE_NAME', 'smartclm-poc')}"
            async_engine = create_async_engine(database_url, echo=False)
            
            async with AsyncSession(async_engine) as session:
                try:
                    # 1. Document 메타데이터 구성
                    filename = source_file.name
                    title = parsed_result.get("title", filename)
                    page_count = parsed_result.get("page_count", 0)
                    markdown_content = parsed_result.get("markdown_content", "")
                    html_content = parsed_result.get("html_content", "")
                    
                    # 파일 크기 계산
                    file_size = source_file.stat().st_size if source_file.exists() else 0
                    
                    # docs/ 기준 상대 경로 계산
                    relative_path = source_file.relative_to(DOCS_ROOT)
                    
                    # 2. Document 생성 (실제 파일 경로 정보 사용)
                    document = Document(
                        room_id=None,  # 전역 문서
                        doc_type=doc_type,
                        category=None,
                        processing_status="processing",
                        filename=filename,
                        version=None,
                        s3_bucket="local",  # 로컬 저장 표시
                        s3_key=str(relative_path),  # docs/ 기준 상대 경로
                        pdf_s3_bucket="local",
                        pdf_s3_key=f"processed/{doc_type}/{filename}",  # 처리된 결과 경로
                        file_size=file_size,
                        pdf_file_size=file_size,  # PDF와 원본이 같다고 가정
                        page_count=page_count,
                        document_metadata={
                            "source": "rag_pipeline",
                            "local_processing": True,
                            "original_path": str(source_file),
                            "processing_time": parsed_result.get("processing_time", 0),
                            "chunks_count": len(parsed_result.get("chunks", [])),
                        },
                        auto_tags=[doc_type],
                        html_content=html_content,
                        markdown_content=markdown_content,
                    )
                    
                    # 3. Document 저장
                    session.add(document)
                    await session.commit()
                    await session.refresh(document)
                    document_id = document.id  # ID를 미리 저장
                    logger.info(f"📄 Document 저장 완료: ID {document_id}")
                    
                    # 4. 청킹 및 임베딩 처리
                    if markdown_content:
                        logger.info(f"✂️ 청킹 및 임베딩 시작: {filename}")
                        
                        chunker = HierarchicalChunker()
                        embedding_service = TitanEmbeddingService()
                        
                        # 마크다운 청킹
                        chunking_result = chunker.chunk_markdown(
                            markdown_content=markdown_content,
                            filename=filename,
                        )
                        
                        # 벡터용 청크 생성
                        vector_ready_chunks = chunker.create_vector_ready_chunks(chunking_result)
                        
                        # 임베딩 생성
                        embedded_chunks = await embedding_service.embed_chunked_documents(vector_ready_chunks)
                        
                        # 청크 저장
                        chunk_objects = []
                        for i, chunk_data in enumerate(embedded_chunks):
                            if chunk_data.get("content", "").strip():
                                chunk = Chunk(
                                    document_id=document.id or 0,
                                    content=chunk_data.get("content", ""),
                                    chunk_index=i,
                                    header_1=chunk_data.get("headers", {}).get("header_1"),
                                    header_2=chunk_data.get("headers", {}).get("header_2"),
                                    header_3=chunk_data.get("headers", {}).get("header_3"),
                                    header_4=chunk_data.get("headers", {}).get("header_4"),
                                    parent_id=chunk_data.get("parent_id"),
                                    child_id=chunk_data.get("child_id"),
                                    chunk_type=chunk_data.get("chunk_type", "child"),
                                    embedding=chunk_data.get("embedding"),
                                    word_count=len(chunk_data.get("content", "").split()),
                                    char_count=len(chunk_data.get("content", "")),
                                    chunk_metadata=chunk_data.get("metadata", {}),
                                    auto_tags=chunk_data.get("auto_tags", []),
                                )
                                chunk_objects.append(chunk)
                        
                        # 배치로 청크 저장
                        for chunk in chunk_objects:
                            session.add(chunk)
                        
                        await session.commit()
                        logger.info(f"✅ 청크 저장 완료: {len(chunk_objects)}개")
                    
                    # 5. 처리 완료 상태로 변경
                    document.processing_status = "completed"
                    session.add(document)
                    await session.commit()
                    
                    logger.info(f"✅ 데이터베이스 저장 완료: {filename} (Document ID: {document_id})")
                    return True
                    
                except Exception as e:
                    await session.rollback()
                    raise e
                    
        except Exception as e:
            logger.error(f"❌ 데이터베이스 저장 오류: {str(e)}")
            import traceback
            logger.error(f"상세 오류: {traceback.format_exc()}")
            return False
    
    # 메서드를 클래스에 추가
    client_class.save_to_database = save_to_database
    return client_class

# ExternalServiceClient에 메서드 추가
ExternalServiceClient = add_save_method(ExternalServiceClient)
print("✅ ExternalServiceClient save_to_database 메서드 추가 완료")


In [ ]:
# RAGPipeline 클래스 정의

class RAGPipeline:
    """RAG 파이프라인 메인 클래스"""
    
    def __init__(self):
        self.client = ExternalServiceClient()
        self.results: List[ProcessingResult] = []
    
    def find_all_documents(self) -> List[Path]:
        """근거 자료 폴더에서만 문서 파일 찾기"""
        supported_extensions = {'.pdf', '.docx', '.doc', '.hwp', '.xlsx', '.xls', '.pptx', '.ppt'}
        document_files = []
        
        # 근거 자료 폴더만 검색
        base_folder = DOCS_ROOT / "근거 자료"
        if base_folder.exists():
            for ext in supported_extensions:
                # 모든 하위 폴더 검색 (법령, 체결계약, 표준계약서)
                document_files.extend(base_folder.glob(f"**/*{ext}"))
        
        return sorted(set(document_files))  # 중복 제거 및 정렬

    async def step1_store_reference_materials(self, force: bool = False):
        """Step 1: 근거자료 저장"""
        logger.info("🚀 Step 1: 근거자료 저장 시작")
        
        # 서비스 헬스체크
        health_status = await self.client.health_check_all()
        if not all(health_status.values()):
            unhealthy_services = [k for k, v in health_status.items() if not v]
            logger.error(f"❌ 다음 서비스들이 비정상입니다: {unhealthy_services}")
            return False
        
        # 문서 파일 목록 가져오기
        document_files = self.find_all_documents()
        if not document_files:
            logger.warning("⚠️ docs 폴더에 지원되는 문서 파일이 없습니다.")
            return False
        
        logger.info(f"📁 발견된 문서 파일: {len(document_files)}개")
        
        # 문서 타입별 분류 출력
        type_counts = {}
        for file_path in document_files:
            doc_type = self.client.determine_doc_type(file_path)
            type_counts[doc_type] = type_counts.get(doc_type, 0) + 1
        
        logger.info("📊 문서 타입별 분류:")
        for doc_type, count in type_counts.items():
            logger.info(f"  - {doc_type}: {count}개")
        
        # 각 문서 파일 처리
        for i, file_path in enumerate(document_files, 1):
            doc_type = self.client.determine_doc_type(file_path)
            logger.info(f"📄 처리 중 ({i}/{len(document_files)}): {file_path.name} ({doc_type})")
            
            # 1. JSON 파일이 있는지 확인 (문서 분석 건너뛰기용)
            doc_type_folder = PROCESSED_DIR / doc_type
            result_file = doc_type_folder / f"{file_path.stem}_parsed.json"
            old_format_file = PROCESSED_DIR / f"{file_path.stem}_{doc_type}_parsed.json"
            
            if not force and (result_file.exists() or old_format_file.exists()):
                # 기존 JSON 파일 로드
                json_file = result_file if result_file.exists() else old_format_file
                logger.info(f"📄 기존 분석 결과 사용: {json_file.name}")
                
                with open(json_file, 'r', encoding='utf-8') as f:
                    doc_parser_result = json.load(f)
                
                result = ProcessingResult(
                    filename=file_path.name,
                    doc_type=doc_type,
                    folder_path=str(file_path.parent),
                    success=True,
                    processing_time=0,
                    page_count=doc_parser_result.get("page_count"),
                    chunk_count=len(doc_parser_result.get("chunks", [])),
                    doc_parser_result=doc_parser_result
                )
            else:
                # 새로 문서 분석
                logger.info(f"🔍 문서 분석 시작: {file_path.name}")
                result = await self.client.analyze_document_file(file_path)
            
            self.results.append(result)
            
            if result.success:
                # 2. 타입별 폴더 생성
                import shutil
                doc_type_folder = PROCESSED_DIR / doc_type
                doc_type_folder.mkdir(exist_ok=True)
                
                # 3. 분석 결과 저장 (타입별 폴더에 JSON 저장)
                result_file = doc_type_folder / f"{file_path.stem}_parsed.json"
                with open(result_file, 'w', encoding='utf-8') as f:
                    json.dump(result.doc_parser_result, f, ensure_ascii=False, indent=2)
                logger.info(f"📄 JSON 저장 완료: {result_file}")
                
                # 4. 원본 파일을 타입별 폴더로 복사
                copied_file = doc_type_folder / file_path.name
                shutil.copy2(file_path, copied_file)
                logger.info(f"📁 파일 복사 완료: {copied_file}")
                
                # 5. 데이터베이스 저장 (중복 체크)
                db_already_exists = False
                if not force:
                    try:
                        from sqlmodel.ext.asyncio.session import AsyncSession
                        from sqlalchemy.ext.asyncio import create_async_engine
                        from src.models import Document
                        from sqlmodel import select
                        
                        database_url = os.getenv("DATABASE_URL") or f"postgresql+asyncpg://{os.getenv('DATABASE_USER', 'postgres')}:{os.getenv('DATABASE_PASSWORD', 'postgres')}@{os.getenv('DATABASE_HOST', 'localhost')}:{os.getenv('DATABASE_PORT', '5434')}/{os.getenv('DATABASE_NAME', 'smartclm-poc')}"
                        async_engine = create_async_engine(database_url, echo=False)
                        
                        async with AsyncSession(async_engine) as session:
                            query = select(Document).where(
                                Document.filename == file_path.name,
                                Document.doc_type == doc_type,
                                Document.processing_status == "completed"
                            )
                            result_doc = await session.exec(query)
                            existing_doc = result_doc.first()
                            
                            if existing_doc:
                                logger.info(f"💾 DB 저장 건너뛰기: {file_path.name} (이미 저장됨: ID {existing_doc.id})")
                                db_already_exists = True
                    except Exception as e:
                        logger.warning(f"⚠️ DB 중복 확인 실패: {file_path.name} - {str(e)}")
                        logger.info(f"💾 중복 확인 실패로 인해 저장을 시도합니다: {file_path.name}")
                
                if not db_already_exists:
                    db_success = await self.client.save_to_database(
                        result.doc_parser_result, 
                        file_path,
                        doc_type
                    )
                    
                    if db_success:
                        logger.info(f"✅ 완료: {file_path.name} ({result.processing_time:.2f}초)")
                    else:
                        logger.error(f"❌ DB 저장 실패: {file_path.name}")
                else:
                    logger.info(f"✅ 완료: {file_path.name} ({result.processing_time:.2f}초) - DB는 기존 사용")
            else:
                logger.error(f"❌ 처리 실패: {file_path.name} - {result.error_message}")
            
            # 처리 간 잠시 대기 (시스템 부하 방지)
            await asyncio.sleep(1)
        
        # 결과 요약 저장
        await self._save_processing_summary()
        
        return True

    async def step2_test_search(self):
        """Step 2: 검색 테스트"""
        logger.info("🔍 Step 2: 검색 테스트 시작")
        
        # 테스트 쿼리 정의
        test_queries = [
            # 법령 관련 쿼리
            {
                "query": "개인정보보호법의 주요 내용은?",
                "category": "law",
                "expected_doc_type": "law"
            },
            {
                "query": "전자상거래 소비자보호에 관한 법률",
                "category": "law", 
                "expected_doc_type": "law"
            },
            # 계약서 관련 쿼리
            {
                "query": "계약해지 조건과 절차",
                "category": "contract",
                "expected_doc_type": ["executed_contract", "standard_contract"]
            },
            {
                "query": "손해배상 책임 범위",
                "category": "contract",
                "expected_doc_type": ["executed_contract", "standard_contract"]
            },
            {
                "query": "계약 기간 및 갱신 조건",
                "category": "contract", 
                "expected_doc_type": ["executed_contract", "standard_contract"]
            },
            # 일반적인 질문
            {
                "query": "계약서 작성 시 주의사항",
                "category": "general",
                "expected_doc_type": "any"
            }
        ]
        
        search_results = []
        
        # 각 쿼리에 대해 검색 수행
        for i, test_case in enumerate(test_queries, 1):
            logger.info(f"🔍 검색 테스트 {i}/{len(test_queries)}: {test_case['query']}")
            
            try:
                # 직접 검색 호출
                start_time = time.time()
                result = await self.client.search_documents_direct(
                    query=test_case["query"],
                    top_k=5,
                    doc_types=None  # 모든 문서 타입에서 검색
                )
                search_time = time.time() - start_time
                
                # 결과 분석
                search_result = {
                    "query": test_case["query"],
                    "category": test_case["category"],
                    "expected_doc_type": test_case["expected_doc_type"],
                    "search_time": search_time,
                    "total_results": len(result.get("results", [])),
                    "results": result.get("results", []),
                    "success": True
                }
                
                # 결과 품질 평가
                if result.get("results"):
                    # 문서 타입 분포 확인
                    doc_types = [r.get("doc_type") for r in result["results"]]
                    search_result["doc_type_distribution"] = {
                        doc_type: doc_types.count(doc_type) 
                        for doc_type in set(doc_types) if doc_type
                    }
                    
                    # 평균 점수 계산
                    scores = [r.get("similarity_score", 0) for r in result["results"]]
                    search_result["avg_similarity_score"] = sum(scores) / len(scores) if scores else 0
                    search_result["max_similarity_score"] = max(scores) if scores else 0
                    search_result["min_similarity_score"] = min(scores) if scores else 0
                
                logger.info(f"  ✅ 검색 완료: {search_result['total_results']}개 결과, {search_time:.2f}초")
                if search_result.get("doc_type_distribution"):
                    for doc_type, count in search_result["doc_type_distribution"].items():
                        logger.info(f"    - {doc_type}: {count}개")
                
            except Exception as e:
                logger.error(f"  ❌ 검색 실패: {str(e)}")
                search_result = {
                    "query": test_case["query"],
                    "category": test_case["category"],
                    "expected_doc_type": test_case["expected_doc_type"],
                    "error": str(e),
                    "success": False
                }
            
            search_results.append(search_result)
            
            # 요청 간 잠시 대기
            await asyncio.sleep(0.5)
        
        # 결과 요약 및 저장
        await self._save_search_test_results(search_results)
        
        logger.info("🎉 Step 2 검색 테스트 완료!")
        return True

print("✅ RAGPipeline 클래스 정의 완료")


In [ ]:
# RAGPipeline 유틸리티 메서드들 추가

def add_rag_utility_methods(rag_class):
    """RAGPipeline에 유틸리티 메서드들 추가"""
    
    async def _save_processing_summary(self):
        """처리 결과 요약 저장"""
        # 타입별 통계
        type_stats = {}
        for result in self.results:
            doc_type = result.doc_type
            if doc_type not in type_stats:
                type_stats[doc_type] = {"total": 0, "success": 0, "failed": 0}
            
            type_stats[doc_type]["total"] += 1
            if result.success:
                type_stats[doc_type]["success"] += 1
            else:
                type_stats[doc_type]["failed"] += 1
        
        summary = {
            "total_files": len(self.results),
            "successful": len([r for r in self.results if r.success]),
            "failed": len([r for r in self.results if not r.success]),
            "total_processing_time": sum(r.processing_time for r in self.results),
            "type_statistics": type_stats,
            "details": [
                {
                    "filename": r.filename,
                    "doc_type": r.doc_type,
                    "folder_path": r.folder_path,
                    "success": r.success,
                    "processing_time": r.processing_time,
                    "page_count": r.page_count,
                    "chunk_count": r.chunk_count,
                    "error": r.error_message
                }
                for r in self.results
            ]
        }
        
        summary_file = RESULTS_DIR / "processing_summary.json"
        with open(summary_file, 'w', encoding='utf-8') as f:
            json.dump(summary, f, ensure_ascii=False, indent=2)
        
        logger.info(f"📊 처리 요약:")
        logger.info(f"  - 전체: {summary['total_files']}개")
        logger.info(f"  - 성공: {summary['successful']}개")
        logger.info(f"  - 실패: {summary['failed']}개")
        logger.info(f"  - 총 처리 시간: {summary['total_processing_time']:.2f}초")
        
        logger.info(f"📋 타입별 통계:")
        for doc_type, stats in type_stats.items():
            logger.info(f"  - {doc_type}: {stats['success']}/{stats['total']} 성공")
        
        logger.info(f"💾 요약 저장: {summary_file}")

    async def _save_search_test_results(self, search_results: List[Dict]):
        """검색 테스트 결과 저장"""
        # 결과 요약 계산
        total_queries = len(search_results)
        successful_queries = len([r for r in search_results if r.get("success", False)])
        failed_queries = total_queries - successful_queries
        
        # 검색 시간 통계
        search_times = [r.get("search_time", 0) for r in search_results if r.get("success")]
        avg_search_time = sum(search_times) / len(search_times) if search_times else 0
        
        # 결과 수 통계
        result_counts = [r.get("total_results", 0) for r in search_results if r.get("success")]
        avg_results = sum(result_counts) / len(result_counts) if result_counts else 0
        
        # 카테고리별 성공률
        category_stats = {}
        for result in search_results:
            category = result.get("category", "unknown")
            if category not in category_stats:
                category_stats[category] = {"total": 0, "success": 0}
            category_stats[category]["total"] += 1
            if result.get("success", False):
                category_stats[category]["success"] += 1
        
        summary = {
            "timestamp": datetime.now().isoformat(),
            "test_summary": {
                "total_queries": total_queries,
                "successful_queries": successful_queries,
                "failed_queries": failed_queries,
                "success_rate": successful_queries / total_queries if total_queries > 0 else 0,
                "avg_search_time_seconds": round(avg_search_time, 3),
                "avg_results_per_query": round(avg_results, 1)
            },
            "category_performance": {
                category: {
                    "success_rate": stats["success"] / stats["total"] if stats["total"] > 0 else 0,
                    "success_count": stats["success"],
                    "total_count": stats["total"]
                }
                for category, stats in category_stats.items()
            },
            "detailed_results": search_results
        }
        
        # 결과 저장
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        results_file = RESULTS_DIR / f"search_test_results_{timestamp}.json"
        
        with open(results_file, 'w', encoding='utf-8') as f:
            json.dump(summary, f, ensure_ascii=False, indent=2)
        
        # 로그 출력
        logger.info("=" * 50)
        logger.info("🔍 검색 테스트 결과 요약")
        logger.info("=" * 50)
        logger.info(f"📊 전체 쿼리: {total_queries}개")
        logger.info(f"✅ 성공: {successful_queries}개 ({successful_queries/total_queries*100:.1f}%)")
        logger.info(f"❌ 실패: {failed_queries}개")
        logger.info(f"⏱️ 평균 검색 시간: {avg_search_time:.3f}초")
        logger.info(f"📄 평균 결과 수: {avg_results:.1f}개")
        
        logger.info("\n📈 카테고리별 성공률:")
        for category, stats in category_stats.items():
            success_rate = stats["success"] / stats["total"] * 100
            logger.info(f"  - {category}: {stats['success']}/{stats['total']} ({success_rate:.1f}%)")
        
        logger.info(f"\n💾 상세 결과 저장: {results_file}")

    # 메서드들을 클래스에 추가
    rag_class._save_processing_summary = _save_processing_summary
    rag_class._save_search_test_results = _save_search_test_results
    return rag_class

# RAGPipeline에 유틸리티 메서드 추가
RAGPipeline = add_rag_utility_methods(RAGPipeline)
print("✅ RAGPipeline 유틸리티 메서드 추가 완료")


In [ ]:
# Step 3: 계약서 검토 테스트 (3단계 체인 방식) 메서드 추가

def add_step3_methods(rag_class):
    """RAGPipeline에 Step 3 관련 메서드들 추가"""
    
    async def step3_contract_review_test(self):
        """Step 3: 계약서 검토 기능 테스트 (3단계 체인 방식)"""
        logger.info("📝 Step 3: 계약서 검토 기능 테스트 시작 (체인 방식)")
        
        # 1. 테스트 대상 문서 선택
        test_document_path = DOCS_ROOT / "1. NDA" / "초안_비밀유지계약서.docx"
        
        if not test_document_path.exists():
            logger.error(f"❌ 테스트 문서가 없습니다: {test_document_path}")
            return False
        
        logger.info(f"📄 테스트 문서: {test_document_path.name}")
        
        # 2. 계약서를 DB에 저장 (중복 체크 포함)
        logger.info("💾 계약서 DB 저장 확인/처리 중...")
        document_id = await self._ensure_contract_in_db(test_document_path)
        
        if not document_id:
            logger.error("❌ 계약서 DB 저장 실패")
            return False
        
        logger.info(f"✅ 계약서 DB 저장 완료 (Document ID: {document_id})")
        
        # 3. DB에서 계약서 정보 조회
        document_info = await self._get_document_from_db(document_id)
        if not document_info:
            logger.error("❌ DB에서 계약서 정보 조회 실패")
            return False
        
        # 4. 계약서 조항 조회
        chunks = await self._get_document_chunks_from_db(document_id)
        parent_chunks = [c for c in chunks if c.get("chunk_type") == "parent"]
        
        logger.info(f"🔍 DB에서 조회한 청크 수: {len(chunks)}")
        logger.info(f"🔍 Parent 청크 수: {len(parent_chunks)}")
        
        if not parent_chunks:
            logger.error("❌ 분석할 조항을 생성할 수 없습니다.")
            return False
        
        logger.info(f"📊 총 {len(parent_chunks)}개 조항 발견")
        
        try:
            # === 3단계 체인 분석 시작 ===
            
            # Chain 1: 전체 계약서 스캔 → 위법 조항 식별
            logger.info("🔍 Chain 1: 전체 계약서 위법 조항 식별 중...")
            start_time = time.time()
            
            violation_candidates = await self._chain1_identify_violations(
                document_name=test_document_path.name,
                all_chunks=parent_chunks,
                document_id=document_id
            )
            
            chain1_time = time.time() - start_time
            logger.info(f"✅ Chain 1 완료: {len(violation_candidates)}개 위법 조항 후보 발견 ({chain1_time:.2f}초)")
            
            if not violation_candidates:
                logger.info("✅ 위법 조항이 발견되지 않았습니다.")
                await self._save_chain_analysis_results(test_document_path.name, {"violations": []}, {
                    "chain1_time": chain1_time,
                    "chain2_time": 0,
                    "chain3_time": 0,
                    "total_time": chain1_time,
                    "violations_found": 0,
                    "total_clauses": len(parent_chunks)
                })
                return True
            
            # Chain 2: 각 위법 조항별 관련 법령 검색 (병렬 처리)
            logger.info("🔍 Chain 2: 위법 조항별 관련 법령 검색 중...")
            start_time = time.time()
            
            violations_with_laws = await self._chain2_search_related_laws(violation_candidates)
            
            chain2_time = time.time() - start_time
            logger.info(f"✅ Chain 2 완료: {len(violations_with_laws)}개 조항의 법령 검색 완료 ({chain2_time:.2f}초)")
            
            # Chain 3: 최종 상세 분석
            logger.info("🔍 Chain 3: 최종 상세 분석 중...")
            start_time = time.time()
            
            final_analysis = await self._chain3_detailed_analysis(
                violations_with_laws=violations_with_laws,
                document_name=test_document_path.name
            )
            
            chain3_time = time.time() - start_time
            logger.info(f"✅ Chain 3 완료: 최종 분석 완료 ({chain3_time:.2f}초)")
            
            # 결과 저장
            await self._save_chain_analysis_results(test_document_path.name, final_analysis, {
                "chain1_time": chain1_time,
                "chain2_time": chain2_time, 
                "chain3_time": chain3_time,
                "total_time": chain1_time + chain2_time + chain3_time,
                "violations_found": len(final_analysis.get("violations", [])),
                "total_clauses": len(parent_chunks)
            })
            
            logger.info("🎉 Step 3 계약서 검토 테스트 완료 (체인 방식)!")
            return True
            
        except Exception as e:
            logger.error(f"❌ 체인 분석 실패: {str(e)}")
            return False
    
    # 메서드를 클래스에 추가
    rag_class.step3_contract_review_test = step3_contract_review_test
    return rag_class

# RAGPipeline에 Step 3 메서드 추가
RAGPipeline = add_step3_methods(RAGPipeline)
print("✅ RAGPipeline Step 3 메서드 추가 완료")


In [ ]:
# Step 3 체인 분석 헬퍼 메서드들 (Part 1: DB 관련)

def add_chain_helper_methods_part1(rag_class):
    """RAGPipeline에 체인 분석 헬퍼 메서드들 추가 (Part 1: DB 관련)"""
    
    async def _ensure_contract_in_db(self, document_path: Path) -> Optional[int]:
        """계약서가 DB에 없으면 저장하고, 있으면 기존 ID 반환"""
        try:
            from sqlmodel.ext.asyncio.session import AsyncSession
            from sqlalchemy.ext.asyncio import create_async_engine
            from src.models import Document
            from sqlmodel import select
            
            # 데이터베이스 연결
            database_url = os.getenv("DATABASE_URL") or f"postgresql+asyncpg://{os.getenv('DATABASE_USER', 'postgres')}:{os.getenv('DATABASE_PASSWORD', 'postgres')}@{os.getenv('DATABASE_HOST', 'localhost')}:{os.getenv('DATABASE_PORT', '5434')}/{os.getenv('DATABASE_NAME', 'smartclm-poc')}"
            async_engine = create_async_engine(database_url, echo=False)
            
            async with AsyncSession(async_engine) as session:
                # 1. 기존 문서 확인
                query = select(Document).where(
                    Document.filename == document_path.name,
                    Document.doc_type == "contract",
                    Document.processing_status == "completed"
                )
                result = await session.exec(query)
                existing_doc = result.first()
                
                if existing_doc:
                    logger.info(f"🗑️ 기존 계약서 삭제 후 재생성: {document_path.name} (ID: {existing_doc.id})")
                    # 기존 청크들 삭제
                    from sqlmodel import delete
                    from src.models import Chunk
                    chunk_delete_stmt = delete(Chunk).where(Chunk.document_id == existing_doc.id)
                    await session.exec(chunk_delete_stmt)
                    
                    # 기존 문서 삭제
                    doc_delete_stmt = delete(Document).where(Document.id == existing_doc.id)
                    await session.exec(doc_delete_stmt)
                    await session.commit()
                    logger.info(f"✅ 기존 데이터 삭제 완료")
                
                # 2. 새로 분석 및 저장
                logger.info(f"🔍 계약서 분석 및 DB 저장: {document_path.name}")
                doc_data = await self.client.analyze_document_file(document_path)
                
                if not doc_data.success:
                    logger.error(f"❌ 문서 분석 실패: {doc_data.error_message}")
                    return None
                
                logger.info(f"✅ 문서 분석 완료 ({doc_data.processing_time:.2f}초)")
                
                # 3. DB 저장
                db_success = await self.client.save_to_database(
                    doc_data.doc_parser_result,
                    document_path,
                    "contract"  # contract 타입으로 저장
                )
                
                if not db_success:
                    logger.error(f"❌ DB 저장 실패: {document_path.name}")
                    return None
                
                # 4. 저장된 문서 ID 조회
                result2 = await session.exec(query)
                new_doc = result2.first()
                
                if new_doc:
                    logger.info(f"✅ 새 계약서 저장 완료: ID {new_doc.id}")
                    return new_doc.id
                
                return None
                
        except Exception as e:
            logger.error(f"❌ 계약서 DB 처리 실패: {str(e)}")
            return None
    
    async def _get_document_from_db(self, document_id: int) -> Optional[Dict]:
        """DB에서 문서 정보 조회"""
        try:
            from sqlmodel.ext.asyncio.session import AsyncSession
            from sqlalchemy.ext.asyncio import create_async_engine
            from src.models import Document
            from sqlmodel import select
            
            database_url = os.getenv("DATABASE_URL") or f"postgresql+asyncpg://{os.getenv('DATABASE_USER', 'postgres')}:{os.getenv('DATABASE_PASSWORD', 'postgres')}@{os.getenv('DATABASE_HOST', 'localhost')}:{os.getenv('DATABASE_PORT', '5434')}/{os.getenv('DATABASE_NAME', 'smartclm-poc')}"
            async_engine = create_async_engine(database_url, echo=False)
            
            async with AsyncSession(async_engine) as session:
                query = select(Document).where(Document.id == document_id)
                result = await session.exec(query)
                document = result.first()
                
                if document:
                    return {
                        "id": document.id,
                        "filename": document.filename,
                        "doc_type": document.doc_type,
                        "markdown_content": document.markdown_content,
                        "page_count": document.page_count
                    }
                return None
                
        except Exception as e:
            logger.error(f"❌ 문서 조회 실패: {str(e)}")
            return None
    
    async def _get_document_chunks_from_db(self, document_id: int) -> List[Dict]:
        """DB에서 문서의 청크들 조회"""
        try:
            from sqlmodel.ext.asyncio.session import AsyncSession
            from sqlalchemy.ext.asyncio import create_async_engine
            from src.models import Chunk
            from sqlmodel import select
            
            database_url = os.getenv("DATABASE_URL") or f"postgresql+asyncpg://{os.getenv('DATABASE_USER', 'postgres')}:{os.getenv('DATABASE_PASSWORD', 'postgres')}@{os.getenv('DATABASE_HOST', 'localhost')}:{os.getenv('DATABASE_PORT', '5434')}/{os.getenv('DATABASE_NAME', 'smartclm-poc')}"
            async_engine = create_async_engine(database_url, echo=False)
            
            async with AsyncSession(async_engine) as session:
                query = select(Chunk).where(
                    Chunk.document_id == document_id,
                    Chunk.chunk_type == "parent"
                ).order_by(Chunk.chunk_index)
                
                result = await session.exec(query)
                chunks = result.all()
                
                # Chunk 모델을 딕셔너리로 변환
                chunk_list = []
                for chunk in chunks:
                    chunk_dict = {
                        "chunk_index": chunk.chunk_index,
                        "chunk_type": chunk.chunk_type,
                        "content": chunk.content,
                        "header_1": chunk.header_1,
                        "header_2": chunk.header_2,
                        "header_3": chunk.header_3,
                        "chunk_metadata": chunk.chunk_metadata or {}
                    }
                    chunk_list.append(chunk_dict)
                
                logger.info(f"📊 DB에서 {len(chunk_list)}개 parent 청크 조회")
                return chunk_list
                
        except Exception as e:
            logger.error(f"❌ 청크 조회 실패: {str(e)}")
            return []
    
    # 메서드들을 클래스에 추가
    rag_class._ensure_contract_in_db = _ensure_contract_in_db
    rag_class._get_document_from_db = _get_document_from_db
    rag_class._get_document_chunks_from_db = _get_document_chunks_from_db
    return rag_class

# RAGPipeline에 체인 헬퍼 메서드 추가 (Part 1)
RAGPipeline = add_chain_helper_methods_part1(RAGPipeline)
print("✅ RAGPipeline 체인 헬퍼 메서드 추가 완료 (Part 1: DB 관련)")


In [ ]:
# Step 3 체인 분석 헬퍼 메서드들 (Part 2: 체인 분석)

def add_chain_helper_methods_part2(rag_class):
    """RAGPipeline에 체인 분석 메서드들 추가 (Part 2: 체인 분석)"""
    
    async def _chain1_identify_violations(self, document_name: str, all_chunks: List[Dict], document_id: int) -> List[Dict]:
        """Chain 1: 전체 계약서를 스캔하여 위법 조항들을 식별"""
        try:
            # 전체 계약서 구조 생성 (원본 조항 번호 보존)
            contract_structure = []
            for i, chunk in enumerate(all_chunks, 1):
                title = chunk.get("header_1", f"조항 {i}")
                content = chunk.get("content", "")[:200] + "..." if len(chunk.get("content", "")) > 200 else chunk.get("content", "")
                

                # title이 이미 "제X조" 형태인지 확인
                if title.startswith("제") and "조" in title:
                    # 이미 조항 번호가 있으면 그대로 사용
                    contract_structure.append(f"{title}: {content}")
                else:
                    # 조항 번호가 없으면 순서대로 추가
                    contract_structure.append(f"제{i}조 {title}: {content}")
            
            full_contract = "\n\n".join(contract_structure)
            
            # Chain 1 프롬프트
            chain1_prompt = f"""# 📋 (주)비에스지파트너스 관점 계약서 위험 조항 식별

**계약서명:** {document_name}
**총 조항수:** {len(all_chunks)}개
**검토 관점:** **(주)비에스지파트너스 입장에서 불리한 조항 중심 분석**

**전체 계약서 구조:**
```
{full_contract}
```

## 🎯 분석 요구사항

이 계약서를 **(주)비에스지파트너스 입장**에서 검토하여 **비에스지파트너스에게 불리하거나 위험한 조항들**을 모두 찾아주세요.

### 비에스지파트너스 관점 상세 검토 기준:

**🔍 비밀정보 관련 위험:**
1. **정보 보호 범위의 과도한 축소**: "비밀" 표시 누락 시 핵심정보도 보호받지 못하는 조항
2. **일방적 비밀유지 의무**: 비에스지파트너스에게만 과도한 비밀유지 의무 부과
3. **비밀정보 정의의 불균형**: 상대방 정보는 넓게, 비에스지파트너스 정보는 좁게 정의

**⚖️ 책임 및 배상 관련 위험:**
4. **과도한 손해배상**: 비에스지파트너스에게만 과중한 배상책임 부과
5. **배상 한도의 불균형**: 상대방은 무제한, 비에스지파트너스는 제한된 배상
6. **입증책임의 전가**: 비에스지파트너스가 무과실을 입증해야 하는 조항

**🚪 계약 해지 및 권리 관련 위험:**
7. **일방적 해지권**: 상대방에게만 해지권을 부여하는 불평등 조항
8. **권리 행사 제한**: 비에스지파트너스의 정당한 권리(가처분, 손해배상청구 등) 제한
9. **통지 의무의 불균형**: 비에스지파트너스에게만 까다로운 통지 의무 부과

**📋 절차 및 기타 위험:**
10. **관할 법원의 불리함**: 비에스지파트너스에게 불리한 원거리 관할 지정
11. **계약 기간의 불균형**: 의무는 길게, 권리는 짧게 설정
12. **법적 위험**: 강행법규 위반으로 비에스지파트너스가 불이익을 받을 수 있는 조항

**⚠️ 특히 주의깊게 검토할 조항:**
- 비밀정보 정의에서 "표시" 요구사항 (실무상 누락 위험 높음)
- 손해배상 조항의 금액 제한 및 면책 조항
- 계약 위반 시 구제수단 제한 조항
- 준거법 및 관할 조항

## 📝 출력 형식 (JSON)

위법 조항이 있다면 다음 형식으로 출력해주세요:

```json
{{
  "violations_found": true,
  "total_violations": 3,
  "violations": [
    {{
      "clause_number": "제3조",
      "clause_title": "비밀정보의 정의",
      "risk_type": "정보_보호_범위의_과도한_축소",
      "risk_level": "높음",
      "brief_reason": "비밀 표시 누락 시 핵심정보도 보호받지 못해 비에스지파트너스에게 매우 위험"
    }},
    {{
      "clause_number": "제9조", 
      "clause_title": "손해배상",
      "risk_type": "과도한_손해배상_및_권리제한",
      "risk_level": "높음",
      "brief_reason": "비에스지파트너스만 배상 한도 제한되고 가처분 청구권도 박탈당함"
    }},
    {{
      "clause_number": "제8조",
      "clause_title": "계약기간",
      "risk_type": "계약_기간의_불균형",
      "risk_level": "중간", 
      "brief_reason": "비에스지파트너스의 의무는 3년+1년, 상대방 의무는 계약 종료와 함께 소멸"
    }}
  ]
}}
```

위법 조항이 없다면:
```json
{{
  "violations_found": false,
  "total_violations": 0,
  "violations": []
}}
```

**중요**: 반드시 JSON 형식으로만 출력하고, 추가 설명은 하지 마세요."""

            # AI 호출
            from src.aws.bedrock_service import BedrockService
            bedrock_service = BedrockService()
            
            response = bedrock_service.invoke_model(
                prompt=chain1_prompt,
                max_tokens=2000,
                temperature=0.0
            )
            
            # JSON 파싱
            response_text = response.get("text", "") if isinstance(response, dict) else str(response)
            
            import json
            import re
            
            # JSON 추출 (```json 태그 제거)
            json_match = re.search(r'```json\s*(\{.*?\})\s*```', response_text, re.DOTALL)
            if json_match:
                json_str = json_match.group(1)
            else:
                # JSON 태그 없이 직접 JSON인 경우
                json_str = response_text.strip()
            
            try:
                result = json.loads(json_str)
                violations = result.get("violations", [])
                
                # chunk_index 추가 (조항 번호에서 추출)
                for violation in violations:
                    clause_num = violation.get("clause_number", "")
                    # "제3조" -> 3 추출
                    import re
                    match = re.search(r'제(\d+)조', clause_num)
                    if match:
                        violation["chunk_index"] = int(match.group(1)) - 1  # 0-based
                    else:
                        violation["chunk_index"] = 0
                
                logger.info(f"🔍 Chain 1 결과: {len(violations)}개 위법 조항 후보 식별")
                return violations
                
            except json.JSONDecodeError as e:
                logger.error(f"❌ Chain 1 JSON 파싱 실패: {e}")
                logger.error(f"응답 텍스트: {response_text[:500]}...")
                return []
                
        except Exception as e:
            logger.error(f"❌ Chain 1 실패: {str(e)}")
            return []
    
    async def _chain2_search_related_laws(self, violation_candidates: List[Dict]) -> List[Dict]:
        """Chain 2: 각 위법 조항별로 관련 법령을 병렬 검색"""
        violations_with_laws = []
        
        # 병렬 처리를 위한 태스크 생성
        search_tasks = []
        for violation in violation_candidates:
            search_query = f"{violation.get('clause_title', '')} {violation.get('brief_reason', '')}"
            task = self._search_laws_for_violation(violation, search_query)
            search_tasks.append(task)
        
        # 병렬 실행
        if search_tasks:
            results = await asyncio.gather(*search_tasks, return_exceptions=True)
            
            for result in results:
                if isinstance(result, Exception):
                    logger.error(f"❌ 법령 검색 실패: {result}")
                else:
                    violations_with_laws.append(result)
        
        logger.info(f"🔍 Chain 2 결과: {len(violations_with_laws)}개 조항의 법령 검색 완료")
        return violations_with_laws
    
    async def _search_laws_for_violation(self, violation: Dict, search_query: str) -> Dict:
        """개별 위법 조항에 대한 법령 검색"""
        try:
            # 관련 법령 검색
            legal_docs = await self.client.search_documents_direct(
                query=search_query,
                top_k=3,
                doc_types=["law"]
            )
            
            violation_with_laws = violation.copy()
            violation_with_laws["related_laws"] = legal_docs.get("results", [])
            violation_with_laws["laws_found"] = len(legal_docs.get("results", []))
            
            logger.info(f"  📖 {violation.get('clause_title', '')}: {len(legal_docs.get('results', []))}개 관련 법령 발견")
            return violation_with_laws
            
        except Exception as e:
            logger.error(f"❌ 법령 검색 실패 ({violation.get('clause_title', '')}): {e}")
            violation_with_laws = violation.copy()
            violation_with_laws["related_laws"] = []
            violation_with_laws["laws_found"] = 0
            return violation_with_laws
    
    # 메서드들을 클래스에 추가
    rag_class._chain1_identify_violations = _chain1_identify_violations
    rag_class._chain2_search_related_laws = _chain2_search_related_laws
    rag_class._search_laws_for_violation = _search_laws_for_violation
    return rag_class

# RAGPipeline에 체인 헬퍼 메서드 추가 (Part 2)
RAGPipeline = add_chain_helper_methods_part2(RAGPipeline)
print("✅ RAGPipeline 체인 헬퍼 메서드 추가 완료 (Part 2: Chain 1, 2)")


In [ ]:
# Step 3 체인 분석 헬퍼 메서드들 (Part 3: Chain 3 및 결과 저장)

def add_chain_helper_methods_part3(rag_class):
    """RAGPipeline에 체인 분석 메서드들 추가 (Part 3: Chain 3 및 결과 저장)"""
    
    async def _chain3_detailed_analysis(self, violations_with_laws: List[Dict], document_name: str) -> Dict:
        """Chain 3: 위법 조항과 법령 근거를 바탕으로 최종 상세 분석"""
        try:
            if not violations_with_laws:
                return {"violations": []}
            
            final_violations = []
            
            for violation_data in violations_with_laws:
                try:
                    # 상세 분석을 위한 프롬프트 구성
                    clause_title = violation_data.get("clause_title", "")
                    clause_number = violation_data.get("clause_number", "")
                    risk_type = violation_data.get("risk_type", "")
                    brief_reason = violation_data.get("brief_reason", "")
                    related_laws = violation_data.get("related_laws", [])
                    
                    # 관련 법령 텍스트 구성
                    laws_text = ""
                    if related_laws:
                        law_descriptions = []
                        for i, law in enumerate(related_laws, 1):
                            filename = law.get('filename', '').replace('.pdf', '')
                            content = law.get('content', '')[:300] + "..." if len(law.get('content', '')) > 300 else law.get('content', '')
                            similarity = f"(유사도: {law.get('similarity_score', 0):.3f})"
                            law_descriptions.append(f"{i}. {filename} {similarity}\n{content}")
                        laws_text = "\n\n".join(law_descriptions)
                    else:
                        laws_text = "관련 법령을 찾을 수 없습니다."
                    
                    chain3_prompt = f"""# 🏛️ (주)비에스지파트너스 관점 계약서 조항 상세 분석

**분석 대상 조항:** {clause_number} {clause_title}
**검토 관점:** (주)비에스지파트너스 입장에서의 위험성 평가
**예비 위험 유형:** {risk_type}
**예비 판단:** {brief_reason}

## 📚 관련 법령 근거

{laws_text}

## 🎯 상세 분석 요구사항

위 조항과 관련 법령을 바탕으로 **(주)비에스지파트너스 입장에서** 다음 형식으로 상세 분석해주세요:

### 출력 형식 (JSON):
```json
{{
  "조항_위치": "{clause_number} ({clause_title})",
  "리스크_유형": "비에스지파트너스 관점의 구체적인 위험 유형 (예: 정보 보호 범위의 과도한 축소, 과도한_배상책임, 불리한_해지조건)",
  "판단_근거": "비에스지파트너스에게 어떤 불이익이 있는지 관련 법령과 함께 구체적 제시 (예: 비에스지파트너스에게만 민법 제398조 위반 수준의 과도한 배상책임 부과)",
  "의견": "비에스지파트너스 입장에서의 구체적인 개선 방안 (예: 상호 배상책임으로 변경하거나 비에스지파트너스 배상 한도 설정 필요)",
  "관련_법령": ["구체적인 법령 조항명"]
}}
```

**중요**: 
1. 반드시 JSON 형식으로만 출력
2. **(주)비에스지파트너스 입장**에서 불리한 점을 중심으로 분석
3. 관련 법령이 있다면 구체적인 조항명 명시  
4. 비에스지파트너스에게 실무적으로 도움이 되는 개선방안 제시
5. 추가 설명 없이 JSON만 출력"""

                    # AI 호출
                    from src.aws.bedrock_service import BedrockService
                    bedrock_service = BedrockService()
                    
                    response = bedrock_service.invoke_model(
                        prompt=chain3_prompt,
                        max_tokens=1500,
                        temperature=0.0
                    )
                    
                    # JSON 파싱
                    response_text = response.get("text", "") if isinstance(response, dict) else str(response)
                    
                    import json
                    import re
                    
                    # JSON 추출
                    json_match = re.search(r'```json\s*(\{.*?\})\s*```', response_text, re.DOTALL)
                    if json_match:
                        json_str = json_match.group(1)
                    else:
                        json_str = response_text.strip()
                    
                    try:
                        detailed_analysis = json.loads(json_str)
                        final_violations.append(detailed_analysis)
                        logger.info(f"  ✅ {clause_title} 상세 분석 완료")
                        
                    except json.JSONDecodeError as e:
                        logger.error(f"❌ Chain 3 JSON 파싱 실패 ({clause_title}): {e}")
                        # 실패 시 기본 형식으로 대체
                        final_violations.append({
                            "조항_위치": f"{clause_number} ({clause_title})",
                            "리스크_유형": risk_type,
                            "판단_근거": brief_reason,
                            "의견": "상세 분석 실패로 인해 기본 정보만 제공",
                            "관련_법령": [law.get('filename', '').replace('.pdf', '') for law in related_laws[:2]]
                        })
                
                except Exception as e:
                    logger.error(f"❌ 개별 조항 분석 실패 ({violation_data.get('clause_title', '')}): {e}")
                    continue
            
            return {
                "contract_analysis": {
                    "document_name": document_name,
                    "total_violations": len(final_violations),
                    "analysis_method": "3_chain_analysis",
                    "analysis_date": datetime.now().isoformat()
                },
                "violations": final_violations
            }
            
        except Exception as e:
            logger.error(f"❌ Chain 3 실패: {str(e)}")
            return {"violations": []}
    
    async def _save_chain_analysis_results(self, document_name: str, analysis_result: Dict, performance_stats: Dict):
        """체인 분석 결과 저장"""
        try:
            # 타임스탬프 생성
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            
            # 결과 파일명
            result_filename = f"chain_analysis_{document_name.replace('.docx', '')}_{timestamp}.json"
            result_file = RESULTS_DIR / result_filename
            
            # 최종 결과 구성
            final_result = {
                "metadata": {
                    "timestamp": datetime.now().isoformat(),
                    "document_name": document_name,
                    "analysis_method": "3_chain_analysis",
                    "performance": performance_stats
                },
                "summary": {
                    "violations_found": len(analysis_result.get("violations", [])),
                    "total_analysis_time": performance_stats.get("total_time", 0),
                    "efficiency_gain": f"기존 조항별 분석 대비 {performance_stats.get('total_clauses', 0) * 30 - performance_stats.get('total_time', 0):.1f}초 단축"
                },
                "analysis_result": analysis_result
            }
            
            # JSON 파일 저장
            with open(result_file, 'w', encoding='utf-8') as f:
                json.dump(final_result, f, ensure_ascii=False, indent=2)
            
            # 로그 출력
            logger.info("=" * 50)
            logger.info("📝 체인 분석 결과 요약")
            logger.info("=" * 50)
            logger.info(f"📄 문서명: {document_name}")
            logger.info(f"🔍 위법 조항 발견: {len(analysis_result.get('violations', []))}개")
            logger.info(f"⏱️ 전체 분석 시간: {performance_stats.get('total_time', 0):.2f}초")
            logger.info(f"   - Chain 1 (위법 조항 식별): {performance_stats.get('chain1_time', 0):.2f}초")
            logger.info(f"   - Chain 2 (법령 검색): {performance_stats.get('chain2_time', 0):.2f}초")  
            logger.info(f"   - Chain 3 (상세 분석): {performance_stats.get('chain3_time', 0):.2f}초")
            
            # 개별 위법 조항 요약
            if analysis_result.get("violations"):
                logger.info(f"📊 발견된 위법 조항:")
                for i, violation in enumerate(analysis_result["violations"], 1):
                    logger.info(f"   {i}. {violation.get('조항_위치', '')} - {violation.get('리스크_유형', '')}")
            
            logger.info(f"💾 결과 저장: {result_file}")
            logger.info("=" * 50)
            
        except Exception as e:
            logger.error(f"❌ 체인 분석 결과 저장 실패: {str(e)}")
    
    # 메서드들을 클래스에 추가
    rag_class._chain3_detailed_analysis = _chain3_detailed_analysis
    rag_class._save_chain_analysis_results = _save_chain_analysis_results
    return rag_class

# RAGPipeline에 체인 헬퍼 메서드 추가 (Part 3)
RAGPipeline = add_chain_helper_methods_part3(RAGPipeline)
print("✅ RAGPipeline 체인 헬퍼 메서드 추가 완료 (Part 3: Chain 3 및 결과 저장)")


In [ ]:
# 파이프라인 인스턴스 생성 및 환경 확인

# 비동기 실행을 위한 nest_asyncio 적용
import nest_asyncio
nest_asyncio.apply()

# 파이프라인 인스턴스 생성
pipeline = RAGPipeline()

print("✅ RAG 파이프라인 인스턴스 생성 완료")
print(f"📁 문서 루트: {DOCS_ROOT}")

# 환경변수 확인
required_env_vars = [
    "DOC_CONVERTER_URL", "DOC_PARSER_URL", "API_URL",
    "DATABASE_URL", "DATABASE_USER", "DATABASE_PASSWORD", 
    "DATABASE_HOST", "DATABASE_PORT", "DATABASE_NAME"
]

print("\n📋 환경변수 확인:")
for var in required_env_vars:
    value = os.getenv(var)
    if value:
        # 비밀번호는 마스킹
        if "PASSWORD" in var:
            value = "*" * len(value)
        print(f"  ✅ {var}: {value}")
    else:
        print(f"  ⚠️ {var}: 설정되지 않음")

# 문서 파일 확인
document_files = pipeline.find_all_documents()
print(f"\n📁 발견된 문서 파일: {len(document_files)}개")

if document_files:
    # 파일 타입별 분류
    type_counts = {}
    for file_path in document_files:
        doc_type = pipeline.client.determine_doc_type(file_path)
        type_counts[doc_type] = type_counts.get(doc_type, 0) + 1
    
    print("📊 문서 타입별 분류:")
    for doc_type, count in type_counts.items():
        print(f"  - {doc_type}: {count}개")
    
    print("\n📄 문서 파일 목록 (처음 5개):")
    for i, file_path in enumerate(document_files[:5], 1):
        doc_type = pipeline.client.determine_doc_type(file_path)
        print(f"  {i}. {file_path.name} ({doc_type})")
    
    if len(document_files) > 5:
        print(f"  ... 및 {len(document_files) - 5}개 파일 더")
else:
    print("⚠️ 처리할 문서 파일이 없습니다.")
    print(f"   다음 폴더에 문서 파일을 넣어주세요: {DOCS_ROOT / '근거 자료'}")


In [ ]:
# Step 1 실행: 근거자료 저장

FORCE_REPROCESS = False  # 강제 재처리 여부 (True로 설정하면 기존 파일도 다시 처리)

print("🚀 Step 1: 근거자료 저장 시작")
print(f"⚙️ 강제 재처리: {'예' if FORCE_REPROCESS else '아니오'}")

try:
    success = await pipeline.step1_store_reference_materials(force=FORCE_REPROCESS)
    
    if success:
        print("\n🎉 Step 1 완료!")
        print(f"📄 처리된 파일: {len(pipeline.results)}개")
        
        # 성공/실패 통계
        successful = len([r for r in pipeline.results if r.success])
        failed = len([r for r in pipeline.results if not r.success])
        
        print(f"✅ 성공: {successful}개")
        print(f"❌ 실패: {failed}개")
        
        if failed > 0:
            print("\n❌ 실패한 파일들:")
            for result in pipeline.results:
                if not result.success:
                    print(f"  - {result.filename}: {result.error_message}")
        
        # 총 처리 시간
        total_time = sum(r.processing_time for r in pipeline.results)
        print(f"\n⏱️ 총 처리 시간: {total_time:.2f}초")
        
        # 타입별 통계
        type_stats = {}
        for result in pipeline.results:
            doc_type = result.doc_type
            if doc_type not in type_stats:
                type_stats[doc_type] = {"total": 0, "success": 0}
            type_stats[doc_type]["total"] += 1
            if result.success:
                type_stats[doc_type]["success"] += 1
        
        print("\n📊 타입별 성공률:")
        for doc_type, stats in type_stats.items():
            success_rate = stats["success"] / stats["total"] * 100
            print(f"  - {doc_type}: {stats['success']}/{stats['total']} ({success_rate:.1f}%)")
        
    else:
        print("❌ Step 1 실패!")
        
except Exception as e:
    print(f"❌ Step 1 실행 중 오류 발생: {str(e)}")
    import traceback
    traceback.print_exc()


In [ ]:
# Step 2 실행: 검색 테스트

print("🔍 Step 2: 검색 테스트 시작")

try:
    success = await pipeline.step2_test_search()
    
    if success:
        print("🎉 Step 2 검색 테스트 완료!")
    else:
        print("❌ Step 2 실패!")
        
except Exception as e:
    print(f"❌ Step 2 실행 중 오류 발생: {str(e)}")
    import traceback
    traceback.print_exc()


In [ ]:
# 개별 검색 테스트 (원하는 쿼리로 직접 테스트)

SEARCH_QUERY = "개인정보보호법"  # 원하는 검색어로 변경
TOP_K = 5  # 반환할 결과 수
DOC_TYPES = None  # 특정 문서 타입만 검색하려면 ["law", "standard_contract"] 등으로 설정

print(f"🔍 개별 검색 테스트")
print(f"검색어: '{SEARCH_QUERY}'")
print(f"결과 수: {TOP_K}개")
print(f"문서 타입 필터: {DOC_TYPES if DOC_TYPES else '모든 타입'}")

try:
    # 검색 실행
    start_time = time.time()
    result = await pipeline.client.search_documents_direct(
        query=SEARCH_QUERY,
        top_k=TOP_K,
        doc_types=DOC_TYPES
    )
    search_time = time.time() - start_time
    
    print(f"\n⏱️ 검색 시간: {search_time:.3f}초")
    print(f"📊 검색 결과: {len(result.get('results', []))}개")
    
    # 결과 출력
    for i, doc in enumerate(result.get("results", []), 1):
        print(f"\n📄 {i}. {doc.get('filename', 'Unknown')}")
        print(f"   📂 타입: {doc.get('doc_type', 'Unknown')}")
        print(f"   🎯 유사도: {doc.get('similarity_score', 0):.4f}")
        print(f"   📝 내용 미리보기:")
        content = doc.get('content', '')[:200] + "..." if len(doc.get('content', '')) > 200 else doc.get('content', '')
        print(f"   {content}")
    
    if not result.get("results"):
        print("❌ 검색 결과가 없습니다.")
        
except Exception as e:
    print(f"❌ 검색 실패: {str(e)}")
    import traceback
    traceback.print_exc()


In [ ]:
# Step 3 실행: 계약서 검토 테스트 (3단계 체인 방식)

print("📝 Step 3: 계약서 검토 테스트 시작")

try:
    success = await pipeline.step3_contract_review_test()
    
    if success:
        print("🎉 Step 3 계약서 검토 테스트 완료!")
    else:
        print("❌ Step 3 실패!")
        
except Exception as e:
    print(f"❌ Step 3 실행 중 오류 발생: {str(e)}")
    import traceback
    traceback.print_exc()


In [ ]:
# 결과 분석 및 시각화

import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

# 한글 폰트 설정 (시스템에 따라 조정 필요)
plt.rcParams['font.family'] = ['DejaVu Sans', 'Malgun Gothic', 'NanumGothic']
plt.rcParams['axes.unicode_minus'] = False

print("📊 결과 분석 시작")

# 1. 처리 결과 요약 파일 확인
summary_file = RESULTS_DIR / "processing_summary.json"
search_results_files = list(RESULTS_DIR.glob("search_test_results_*.json"))
chain_results_files = list(RESULTS_DIR.glob("chain_analysis_*.json"))

if summary_file.exists():
    # 처리 결과 분석
    with open(summary_file, 'r', encoding='utf-8') as f:
        summary = json.load(f)
    
    print(f"\n📄 문서 처리 결과 요약:")
    print(f"  전체 파일: {summary['total_files']}개")
    print(f"  성공: {summary['successful']}개")
    print(f"  실패: {summary['failed']}개")
    print(f"  총 처리 시간: {summary['total_processing_time']:.2f}초")
    
    # 문서 타입별 성공률 시각화
    if summary.get('type_statistics'):
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
        
        # 타입별 파일 수
        doc_types = list(summary['type_statistics'].keys())
        totals = [summary['type_statistics'][dt]['total'] for dt in doc_types]
        
        ax1.bar(doc_types, totals, color='skyblue')
        ax1.set_title('문서 타입별 파일 수')
        ax1.set_xlabel('문서 타입')
        ax1.set_ylabel('파일 수')
        ax1.tick_params(axis='x', rotation=45)
        
        # 타입별 성공률
        success_rates = [summary['type_statistics'][dt]['success'] / summary['type_statistics'][dt]['total'] * 100 
                        for dt in doc_types]
        
        ax2.bar(doc_types, success_rates, color='lightgreen')
        ax2.set_title('문서 타입별 성공률')
        ax2.set_xlabel('문서 타입')
        ax2.set_ylabel('성공률 (%)')
        ax2.set_ylim(0, 100)
        ax2.tick_params(axis='x', rotation=45)
        
        plt.tight_layout()
        plt.show()
        
        # 상세 통계 출력
        print(f"\n📊 문서 타입별 통계:")
        for doc_type, stats in summary['type_statistics'].items():
            success_rate = stats['success'] / stats['total'] * 100
            print(f"  {doc_type}: {stats['success']}/{stats['total']} ({success_rate:.1f}%)")
    
else:
    print("⚠️ 처리 결과 요약 파일이 없습니다. Step 1을 먼저 실행해주세요.")

# 2. 검색 결과 분석
if search_results_files:
    # 가장 최근 검색 결과 파일 사용
    latest_search_file = sorted(search_results_files)[-1]
    
    with open(latest_search_file, 'r', encoding='utf-8') as f:
        search_data = json.load(f)
    
    print(f"\n🔍 검색 테스트 결과 요약:")
    test_summary = search_data['test_summary']
    print(f"  전체 쿼리: {test_summary['total_queries']}개")
    print(f"  성공: {test_summary['successful_queries']}개")
    print(f"  실패: {test_summary['failed_queries']}개")
    print(f"  성공률: {test_summary['success_rate']*100:.1f}%")
    print(f"  평균 검색 시간: {test_summary['avg_search_time_seconds']:.3f}초")
    print(f"  평균 결과 수: {test_summary['avg_results_per_query']:.1f}개")
    
    # 카테고리별 성공률 시각화
    if search_data.get('category_performance'):
        categories = list(search_data['category_performance'].keys())
        success_rates = [search_data['category_performance'][cat]['success_rate'] * 100 
                        for cat in categories]
        
        plt.figure(figsize=(8, 5))
        plt.bar(categories, success_rates, color='orange')
        plt.title('검색 카테고리별 성공률')
        plt.xlabel('카테고리')
        plt.ylabel('성공률 (%)')
        plt.ylim(0, 100)
        plt.tick_params(axis='x', rotation=45)
        plt.tight_layout()
        plt.show()
        
        print(f"\n📈 카테고리별 성공률:")
        for category, performance in search_data['category_performance'].items():
            success_rate = performance['success_rate'] * 100
            print(f"  {category}: {performance['success_count']}/{performance['total_count']} ({success_rate:.1f}%)")
    
else:
    print("⚠️ 검색 결과 파일이 없습니다. Step 2를 먼저 실행해주세요.")

# 3. 체인 분석 결과
if chain_results_files:
    latest_chain_file = sorted(chain_results_files)[-1]
    
    with open(latest_chain_file, 'r', encoding='utf-8') as f:
        chain_data = json.load(f)
    
    print(f"\n📝 체인 분석 결과 요약:")
    summary_data = chain_data['summary']
    performance = chain_data['metadata']['performance']
    
    print(f"  문서명: {chain_data['metadata']['document_name']}")
    print(f"  위법 조항 발견: {summary_data['violations_found']}개")
    print(f"  전체 분석 시간: {summary_data['total_analysis_time']:.2f}초")
    print(f"  Chain 1 시간: {performance['chain1_time']:.2f}초")
    print(f"  Chain 2 시간: {performance['chain2_time']:.2f}초")
    print(f"  Chain 3 시간: {performance['chain3_time']:.2f}초")
    
    # 체인별 시간 분포 시각화
    chain_times = [performance['chain1_time'], performance['chain2_time'], performance['chain3_time']]
    chain_labels = ['Chain 1\n(위법조항 식별)', 'Chain 2\n(법령 검색)', 'Chain 3\n(상세 분석)']
    
    plt.figure(figsize=(8, 5))
    plt.bar(chain_labels, chain_times, color=['red', 'blue', 'green'])
    plt.title('체인별 분석 시간')
    plt.xlabel('분석 단계')
    plt.ylabel('시간 (초)')
    plt.tight_layout()
    plt.show()
    
    # 발견된 위법 조항 요약
    violations = chain_data['analysis_result'].get('violations', [])
    if violations:
        print(f"\n📊 발견된 위법 조항:")
        for i, violation in enumerate(violations, 1):
            print(f"  {i}. {violation.get('조항_위치', '')} - {violation.get('리스크_유형', '')}")
    
else:
    print("⚠️ 체인 분석 결과 파일이 없습니다. Step 3을 먼저 실행해주세요.")

# 4. 결과 파일 목록
print(f"\n📁 생성된 결과 파일들:")
result_files = list(RESULTS_DIR.glob("*"))
for result_file in sorted(result_files):
    file_size = result_file.stat().st_size
    file_size_mb = file_size / (1024 * 1024)
    print(f"  📄 {result_file.name} ({file_size_mb:.2f} MB)")

print(f"\n📊 분석 완료! 결과는 {RESULTS_DIR} 폴더에서 확인할 수 있습니다.")


In [ ]:
# 데이터베이스 상태 확인

async def check_database_status():
    """데이터베이스 상태 및 저장된 문서 확인"""
    try:
        from sqlmodel.ext.asyncio.session import AsyncSession
        from sqlalchemy.ext.asyncio import create_async_engine
        from src.models import Document, Chunk
        from sqlmodel import select, func
        
        database_url = os.getenv("DATABASE_URL") or f"postgresql+asyncpg://{os.getenv('DATABASE_USER', 'postgres')}:{os.getenv('DATABASE_PASSWORD', 'postgres')}@{os.getenv('DATABASE_HOST', 'localhost')}:{os.getenv('DATABASE_PORT', '5434')}/{os.getenv('DATABASE_NAME', 'smartclm-poc')}"
        async_engine = create_async_engine(database_url, echo=False)
        
        async with AsyncSession(async_engine) as session:
            # 문서 수 확인
            doc_count_query = select(func.count(Document.id))
            doc_result = await session.exec(doc_count_query)
            total_docs = doc_result.one()
            
            # 타입별 문서 수
            type_query = select(Document.doc_type, func.count(Document.id)).group_by(Document.doc_type)
            type_result = await session.exec(type_query)
            type_counts = {doc_type: count for doc_type, count in type_result.all()}
            
            # 청크 수 확인
            chunk_count_query = select(func.count(Chunk.id))
            chunk_result = await session.exec(chunk_count_query)
            total_chunks = chunk_result.one()
            
            # 처리 상태별 문서 수
            status_query = select(Document.processing_status, func.count(Document.id)).group_by(Document.processing_status)
            status_result = await session.exec(status_query)
            status_counts = {status: count for status, count in status_result.all()}
            
            print("📊 데이터베이스 상태:")
            print(f"  총 문서 수: {total_docs}개")
            print(f"  총 청크 수: {total_chunks}개")
            
            print("\n📋 문서 타입별 분포:")
            for doc_type, count in type_counts.items():
                print(f"  - {doc_type}: {count}개")
            
            print("\n⚙️ 처리 상태별 분포:")
            for status, count in status_counts.items():
                print(f"  - {status}: {count}개")
            
            # 최근 추가된 문서들
            recent_query = select(Document.filename, Document.doc_type, Document.processing_status).order_by(Document.id.desc()).limit(5)
            recent_result = await session.exec(recent_query)
            recent_docs = recent_result.all()
            
            if recent_docs:
                print("\n🆕 최근 추가된 문서 (최대 5개):")
                for i, (filename, doc_type, status) in enumerate(recent_docs, 1):
                    print(f"  {i}. {filename} ({doc_type}) - {status}")
            
    except Exception as e:
        print(f"❌ 데이터베이스 상태 확인 실패: {str(e)}")

# 실행
print("🔍 데이터베이스 상태 확인 중...")
await check_database_status()


In [ ]:
# 파일 정리 및 유틸리티

def cleanup_temporary_files():
    """임시 파일들 정리"""
    try:
        import shutil
        
        # 변환된 PDF 파일들 정리
        converted_pdfs = list(PROCESSED_DIR.glob("*_converted.pdf"))
        for pdf_file in converted_pdfs:
            pdf_file.unlink()
            print(f"🗑️ 임시 PDF 삭제: {pdf_file.name}")
        
        print(f"✅ 임시 파일 정리 완료: {len(converted_pdfs)}개 파일 삭제")
        
    except Exception as e:
        print(f"❌ 파일 정리 실패: {str(e)}")

def show_results_summary():
    """결과 파일들 요약 출력"""
    print("📁 결과 파일 요약:")
    
    # 처리 요약
    summary_file = RESULTS_DIR / "processing_summary.json"
    if summary_file.exists():
        print(f"  ✅ 처리 요약: {summary_file.name}")
    
    # 검색 테스트 결과
    search_files = list(RESULTS_DIR.glob("search_test_results_*.json"))
    print(f"  🔍 검색 테스트 결과: {len(search_files)}개 파일")
    
    # 체인 분석 결과
    chain_files = list(RESULTS_DIR.glob("chain_analysis_*.json"))
    print(f"  📝 체인 분석 결과: {len(chain_files)}개 파일")
    
    # 처리된 문서들
    processed_folders = [f for f in PROCESSED_DIR.iterdir() if f.is_dir()]
    print(f"  📄 처리된 문서 타입: {len(processed_folders)}개 폴더")
    for folder in processed_folders:
        doc_count = len(list(folder.glob("*")))
        print(f"    - {folder.name}: {doc_count}개 파일")

# 전체 파이프라인 상태 요약
def show_pipeline_status():
    """전체 파이프라인 상태 요약"""
    print("=" * 60)
    print("🎯 Smart CLM RAG 파이프라인 상태 요약")
    print("=" * 60)
    
    # Step 1 상태
    if hasattr(pipeline, 'results') and pipeline.results:
        successful = len([r for r in pipeline.results if r.success])
        total = len(pipeline.results)
        print(f"📄 Step 1 (문서 저장): {successful}/{total} 성공")
    else:
        print("📄 Step 1 (문서 저장): 미실행")
    
    # Step 2 상태
    search_files = list(RESULTS_DIR.glob("search_test_results_*.json"))
    if search_files:
        print(f"🔍 Step 2 (검색 테스트): 완료 ({len(search_files)}개 결과)")
    else:
        print("🔍 Step 2 (검색 테스트): 미실행")
    
    # Step 3 상태
    chain_files = list(RESULTS_DIR.glob("chain_analysis_*.json"))
    if chain_files:
        print(f"📝 Step 3 (계약서 검토): 완료 ({len(chain_files)}개 결과)")
    else:
        print("📝 Step 3 (계약서 검토): 미실행")
    
    print("=" * 60)

# 실행
print("🧹 파일 정리 중...")
cleanup_temporary_files()

print("\n📊 결과 요약:")
show_results_summary()

print("\n📈 파이프라인 상태:")
show_pipeline_status()

print(f"\n🎉 노트북 실행 완료! 모든 결과는 {RESULTS_DIR} 폴더에서 확인할 수 있습니다.")
